<a href="https://colab.research.google.com/github/abagnard/266FinalProject/blob/main/W266_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os, sys, re, csv, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
from scipy import stats
import pandas as pd

import nltk



#FOMC Data Scraper

###Links to Data:
**Start-2014:** https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm
**2015-2020:** https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm

###2010-2020 General Link Format:
  **Statement/Press Release:** https://www.federalreserve.gov/newsevents/pressreleases/monetaryYYYYMMDDa.htm

**Minutes:** https://www.federalreserve.gov/monetarypolicy/fomcminutesYYYYMMDD.htm

In [ ]:
from   bs4    import BeautifulSoup
from   urllib.request import urlopen

def getFOMCStatementURL(date):
  if date >= 20100000:
    url = "https://www.federalreserve.gov/newsevents/pressreleases/monetary" + str(date) + "a.htm"
  return url

def getFOMCMinutesURL(date):
  if date >= 20100000:
    url = "https://www.federalreserve.gov/monetarypolicy/fomcminutes" + str(date) + ".htm"
  return url

def getFOMCStatementData(date):
  html = urlopen(getFOMCStatementURL(date)).read()
  soup = BeautifulSoup(html)
  text = soup.get_text(" ")

  startLocation = re.search("For immediate release", text).start() + len("For immediate release")
  endLocation = re.search("Voting for", text).start()

  statementText = text[startLocation : endLocation]
  return statementText

def getFOMCMinutesData(date):
  html = urlopen(getFOMCMinutesURL(date)).read()
  soup = BeautifulSoup(html)
  text = soup.get_text(" ")

  startLocation = re.search("Minutes of the Federal Open Market Committee", text).start() + len("Minutes of the Federal Open Market Committee")
  endLocation = re.search("Notation Vote", text).start()

  minutesText = text[startLocation : endLocation]
  return minutesText


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


SyntaxError: ignored

In [ ]:
dates = [20100127
,20100316
,20100428]
# for date in dates:
#   print("Statement on ", date)
#   print(getFOMCStatementData(date))


# for date in dates:
#   print("Minutes on ", date)
#   print(getFOMCMinutesData(date))


Pull Dates from Google Sheet

In [ ]:
# Code Source
# https://medium.com/analytics-vidhya/colab-and-google-sheets-surprisingly-powerful-combination-for-data-science-part-1-bbbb11cbd8e

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
date_google_sheet_url = gc.open_by_url("https://docs.google.com/spreadsheets/d/1Q9_JimVaDffFe8U4SYQ_VLU9-1aeuFXlDZhvTsFD0kg/edit#gid=1023419829")
date_google_sheet = date_google_sheet_url.worksheet("Dates")
date_data_raw = date_google_sheet.get_all_values()

df_date = pd.DataFrame(date_data_raw)
df_date.columns = df_date.iloc[0]
df_date = df_date.iloc[1:]

df_date.head()
dates = df_date["Date"].values.tolist()

,Date,PressConference
1,20100127,0
2,20100316,0
3,20100428,0
4,20100623,0
5,20100810,0


In [54]:
def createFOMCStatementTxtFile():
  file_path = "/content/drive/My Drive/W266 - NLP/Final Project/data/FOMC_data/FOMC_Raw/"
  for date in dates:
    data = getFOMCStatementData(date)
    filename = "FOMCStatement_" + date + ".txt"
    file = open(file_path+filename, "w")
    file.write(data)
    file.close

In [55]:
from google.colab import drive
drive.mount('/content/drive')
# cd /content/drive/'My Drive'/'W266 - NLP'/'Final Project'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: ignored

In [56]:
ls /content/drive/"My Drive"/"W266 - NLP"/"Final Project"/data/FOMC_data/FOMC_Raw


Fed Note


Fed Release

Financial News Sources